In [2]:
#GET Data
from google.colab import files
files.upload() #Upload kaggle.json here.

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [1]:
import torch 
from torch import nn
import pandas as pd
import torch.optim as optim 
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms as T
import torch.nn.functional as F

In [3]:
!kaggle datasets download -d zalando-research/fashionmnist
!unzip -q fashionmnist.zip
!rm fashionmnist.zip

 74% 51.0M/68.8M [00:00<00:00, 137MB/s]
100% 68.8M/68.8M [00:00<00:00, 161MB/s]


In [25]:
#Make sets
train_set = pd.read_csv('fashion-mnist_train.csv')
validation_set = train_set.iloc[50000:,].reset_index().drop(['index'], axis=1)
test_set = pd.read_csv('fashion-mnist_test.csv')
train_set = train_set.iloc[:50000]

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=20, shuffle=True, num_workers = 0)
test_loader = DataLoader(dataset=test_set, batch_size=20, shuffle=True, num_workers = 0)

In [26]:
train_set

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2,0,0,0,0,1,0,0,33,176,...,180,174,154,150,10,0,2,1,0,0
49996,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,4,0,0,0,0,0,0,0,0,0,...,189,91,0,120,126,40,0,0,0,0
49998,4,0,0,0,0,0,0,1,2,0,...,3,0,27,136,129,85,0,0,0,0


In [35]:
#Make model
#greyscale so in_channel is 1, out_channel chosen arbitrarily, same with kernel and stride

class CNN(nn.Module):
  #these are the default values, like a constructor
  def __init__(self, in_channels = 3 , out_channels = 100, kernel_size = 4, stride = 1): 
    super(CNN, self).__init__() 
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels=50, kernel_size =4, stride=1) #first layer
    self.b1 = nn.BatchNorm2d(50)
    self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size =2, stride =2)
    self.b2 = nn.BatchNorm2d(10)

    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2)) #will cut output to half size
    #output has to be linear
    self.o = nn.Linear(10 * 6 * 6 , 10)

    # (B, 100, 25, 25) -> (B, 100 * 25 * 25)
    # input to linear: (B, D)
    # Linear(D, out_dim)
    # output from linear: (B, out_dim)

    # input to conv layer: (B, NC, h, w)
    # Conv2D()
    # output from conv layer: (B, out_channels, H_new, W_new)

  #got through all your layers
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.b1(x)
    x = F.relu(self.conv2(x))
    x = self.b2(x)
    x = self.pool(x)

    x = x.reshape(x.shape[0], -1)
    x = self.o(x)

    return x

In [36]:
###REFORMAT TRAIN DATA ?
class CustomImageDataset(Dataset):
    def __init__(self, data):
        self.data = data.values
        self.transform = T.Compose([
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data[idx]  # (785,)

        label = torch.Tensor(row[:1]) # (1,)
        img = row[1:].reshape(28, 28) #(784,) -> (28, 28)

        img = torch.Tensor(np.stack([img, img, img], axis=0))  # (3, 28, 28)
        img = self.transform(img)  # (3, 28, 28)

        return img, label

In [37]:
batch_size = 64

train_data  = CustomImageDataset(train_set)

train_loader = DataLoader(train_data, 
                          batch_size=batch_size, 
                          shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CNN().to(device)

num_epochs = 10
learning_rate = 0.001

criterion = nn.CrossEntropyLoss() #Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [38]:
##Training / instantiate some values
for epoch in range(num_epochs):
   for i, batch in enumerate(train_loader):
        data, labels = batch 
        data = data.to(device)
        labels = labels.squeeze().type(torch.LongTensor).to(device)
        # Forward pass
     
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        # Backward and optimize
        loss.backward()

   print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/10], Loss: 2.7000
Epoch [2/10], Loss: 2.5350
Epoch [3/10], Loss: 2.6105
Epoch [4/10], Loss: 2.7517
Epoch [5/10], Loss: 2.6612


KeyboardInterrupt: ignored